# Import des librairies nécessaires

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Sommaire

1. partition du dataset (train, test)
2. best feature
2. kppv